<a href="https://colab.research.google.com/github/wiv33/A-Learning-python/blob/master/machine-learning/_000_hello_machine/_000_basic/_003_cuk_edu/_005_3_computer_vision/09_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Errata

1. 1차시 강의 중 최대 객체 수 오류 정정  
   1차시 30:19 경에 YOLO의 예를 들면서 grid가 19x19이고 Anchor가 9개이고 class가 80개인 경우 27만개라고 한 부분은 잘못된 설명입니다. (27만개는 단순한 텐서의 크기입니다.)  
   감지할 수 있는 총 객체의 수는 = grid x grid x anchor 수 만큼입니다.  
   따라서 한 이미지에서 YOLO의 기본적인 최대 객체 감지 수는 19x19x9 = 3249개 입니다.  

# Batch normalization


gdown upgrade

In [1]:
!pip install --upgrade gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


구글 드라이브 마운트

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive

Mounted at /content/gdrive


**Batch normalization**

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import  Conv2D, MaxPooling2D, Flatten, Dense

model = keras.Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 576)               0

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import BatchNormalization, Activation

model = keras.Sequential([
    Conv2D(32, (3,3), input_shape=(28, 28, 1)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3)),
    BatchNormalization(),
    Activation('relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 11, 11, 64)      

In [8]:
93962-93322

640

In [ ]:
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation

# 기존 
Conv2D(64, (3,3), activation='relu'),

# 기존과 동일 코드
Conv2D(64, (3,3)),
Activation('relu'),

# Batch Normalization
Conv2D(64, (3,3)),
BatchNormalization(),
Activation('relu'),



(<keras.layers.core.Activation at 0x7f39201dfd90>,)

# Keras-YOLO  

공개 소스(open source) 기반의 대규모 소스를 한번에 이해하기는 어렵습니다.  
특히 이번 yolo 소스의 경우에는 전체 소스가 google colab에서 보이지 않습니다.  
전체 소스를 이해하기 위해서는 GIT에서 전체 소스를 다운받아서 천천히 살펴 보아야 합니다.

우선적으로 다음과 같은 순서로 소스를 살펴보도록 합시다.

1. 소스의 구동  
   오픈 소스의 경우 구동 자체가 어려운 경우가 많습니다.  
   차근차근 실행해 보면서 전체의 구동 구조를 파악합니다.

2. 데이터 형식의 이해  
   입출력 데이터 형식을 이해하려고 노력해 봅시다.  
   객체 탐지의 경우 프로젝트마다 유사점은 있지만, 각각의 독립적인 데이터 형식들을 취하고 있습니다.  
   입출력 데이터의 형식을 파악하는 것이 중요합니다.

3. 소스의 이해  
   소스를 파악할 때, 세부사항보다는 전체의 함수 흐름을 먼저 이해해 봅시다.  
   당장은 이해가 안가더라도, 소스를 구동하고 각각의 변수를 출력해가면서 분석하다 보면,  
   점차적으로 이해되는 경우가 많습니다.  

**이번 실습의 목표**  

본 실습의 목표는 소스의 대략적인 구조의 이해와 구동을 성공시키는 것입니다.  
그리고 강의에서 배운 YOLO의 구조들이 소스의 어느 부분에 위치하는 지를 파악해 봅시다.

**Source code repo**  
https://github.com/shevious/keras-yolo4.git  

**Original source code repo**  
https://github.com/Ma-Dan/keras-yolo4  


In [9]:
%cd /content
!git clone https://github.com/shevious/keras-yolo4.git

/content
Cloning into 'keras-yolo4'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 96 (delta 8), reused 12 (delta 4), pack-reused 72
Unpacking objects: 100% (96/96), done.


In [10]:
%cd /content/keras-yolo4

/content/keras-yolo4


In [11]:
import gdown
md5 = '6cd6e144f989b92b3379bac3b3de84fd'
#url = 'https://drive.google.com/file/d/14eyB2lkxrG83fGy4t395sgZqADOYJ_yE/view?usp=sharing'
url = 'https://drive.google.com/uc?id=14eyB2lkxrG83fGy4t395sgZqADOYJ_yE'
output = '/mydrive/VOCtrainval_11-May-2012.tar'
gdown.cached_download(url, output, md5=md5)

Computing MD5: /mydrive/VOCtrainval_11-May-2012.tar
MD5 matches: /mydrive/VOCtrainval_11-May-2012.tar


'/mydrive/VOCtrainval_11-May-2012.tar'

In [12]:
!tar xf /mydrive/VOCtrainval_11-May-2012.tar

In [13]:
!ln -s VOCdevkit data

In [14]:
!python voc_annotation.py

**import section**

In [15]:
from functools import wraps

import math
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Conv2D, Add, ZeroPadding2D, UpSampling2D, Concatenate, MaxPooling2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

from yolo4.utils import compose



**Mish activation function**

In [16]:
class Mish(Layer):
    '''
    Mish Activation Function.
    .. math::
        mish(x) = x * tanh(softplus(x)) = x * tanh(ln(1 + e^{x}))
    Shape:
        - Input: Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
        - Output: Same shape as the input.
    Examples:
        >>> X_input = Input(input_shape)
        >>> X = Mish()(X_input)
    '''

    def __init__(self, **kwargs):
        super(Mish, self).__init__(**kwargs)
        self.supports_masking = True

    def call(self, inputs):
        return inputs * K.tanh(K.softplus(inputs))

    def get_config(self):
        config = super(Mish, self).get_config()
        return config

    def compute_output_shape(self, input_shape):
        return input_shape

**CSPDarknet53 backbone**

In [17]:
@wraps(Conv2D) # comment, document를 그대로 사용하는 의미
def DarknetConv2D(*args, **kwargs):
    """Wrapper to set Darknet parameters for Convolution2D."""
    darknet_conv_kwargs = {}
    darknet_conv_kwargs['kernel_initializer'] = keras.initializers.RandomNormal(mean=0.0, stddev=0.01)
    darknet_conv_kwargs['padding'] = 'valid' if kwargs.get('strides')==(2,2) else 'same'
    darknet_conv_kwargs.update(kwargs)
    return Conv2D(*args, **darknet_conv_kwargs)

def DarknetConv2D_BN_Leaky(*args, **kwargs):
    """Darknet Convolution2D followed by BatchNormalization and LeakyReLU."""
    no_bias_kwargs = {'use_bias': False}
    no_bias_kwargs.update(kwargs)
    return compose(
        DarknetConv2D(*args, **no_bias_kwargs),
        BatchNormalization(),
        LeakyReLU(alpha=0.1)) # alpha = leaky 왼쪽 부분의 기울기

def DarknetConv2D_BN_Mish(*args, **kwargs):
    """Darknet Convolution2D followed by BatchNormalization and LeakyReLU."""
    no_bias_kwargs = {'use_bias': False}
    no_bias_kwargs.update(kwargs)
    return compose(
        DarknetConv2D(*args, **no_bias_kwargs),
        BatchNormalization(),
        Mish())

# 위 함수들을 합친 body
# cross stage partial 
# backbone
def resblock_body(x, num_filters, num_blocks, all_narrow=True):
    '''A series of resblocks starting with a downsampling Convolution2D'''
    # Darknet uses left and top padding instead of 'same' mode
    preconv1 = ZeroPadding2D(((1,0),(1,0)))(x)
    preconv1 = DarknetConv2D_BN_Mish(num_filters, (3,3), strides=(2,2))(preconv1)
    
    shortconv = DarknetConv2D_BN_Mish(num_filters//2 if all_narrow else num_filters, (1,1))(preconv1)
    mainconv = DarknetConv2D_BN_Mish(num_filters//2 if all_narrow else num_filters, (1,1))(preconv1)
    # skip connection
    for i in range(num_blocks):
        y = compose(
                DarknetConv2D_BN_Mish(num_filters//2, (1,1)),
                DarknetConv2D_BN_Mish(num_filters//2 if all_narrow else num_filters, (3,3)))(mainconv)
        mainconv = Add()([mainconv,y])
    postconv = DarknetConv2D_BN_Mish(num_filters//2 if all_narrow else num_filters, (1,1))(mainconv)
    
    route = Concatenate()([postconv, shortconv])
    return DarknetConv2D_BN_Mish(num_filters, (1,1))(route)

def darknet_body(x):
    '''Darknent body having 52 Convolution2D layers'''
    x = DarknetConv2D_BN_Mish(32, (3,3))(x)
    x = resblock_body(x, 64, 1, False)
    x = resblock_body(x, 128, 2)
    x = resblock_body(x, 256, 8)
    x = resblock_body(x, 512, 8)
    x = resblock_body(x, 1024, 4)
    return x

**YOLO body**

In [18]:
# neck, head 로 불리우는 부분
def yolo4_body(inputs, num_anchors, num_classes):
    """Create YOLO_V4 model CNN body in Keras."""
    darknet = Model(inputs, darknet_body(inputs))

    #19x19 head /// 원 이미지의 1/32로 줄었다는 의미
    y19 = DarknetConv2D_BN_Leaky(512, (1,1))(darknet.output)
    y19 = DarknetConv2D_BN_Leaky(1024, (3,3))(y19)
    y19 = DarknetConv2D_BN_Leaky(512, (1,1))(y19)
    # SPP - Spatial Pyramid Pooling
    maxpool1 = MaxPooling2D(pool_size=(13,13), strides=(1,1), padding='same')(y19)
    maxpool2 = MaxPooling2D(pool_size=(9,9), strides=(1,1), padding='same')(y19)
    maxpool3 = MaxPooling2D(pool_size=(5,5), strides=(1,1), padding='same')(y19)
    y19 = Concatenate()([maxpool1, maxpool2, maxpool3, y19])

    y19 = DarknetConv2D_BN_Leaky(512, (1,1))(y19)
    y19 = DarknetConv2D_BN_Leaky(1024, (3,3))(y19)
    y19 = DarknetConv2D_BN_Leaky(512, (1,1))(y19)

    y19_upsample = compose(DarknetConv2D_BN_Leaky(256, (1,1)), UpSampling2D(2))(y19)

    #38x38 head
    y38 = DarknetConv2D_BN_Leaky(256, (1,1))(darknet.layers[204].output)
    y38 = Concatenate()([y38, y19_upsample])
    y38 = DarknetConv2D_BN_Leaky(256, (1,1))(y38)
    y38 = DarknetConv2D_BN_Leaky(512, (3,3))(y38)
    y38 = DarknetConv2D_BN_Leaky(256, (1,1))(y38)
    y38 = DarknetConv2D_BN_Leaky(512, (3,3))(y38)
    y38 = DarknetConv2D_BN_Leaky(256, (1,1))(y38)

    y38_upsample = compose(DarknetConv2D_BN_Leaky(128, (1,1)), UpSampling2D(2))(y38)

    #76x76 head
    y76 = DarknetConv2D_BN_Leaky(128, (1,1))(darknet.layers[131].output)
    y76 = Concatenate()([y76, y38_upsample])
    y76 = DarknetConv2D_BN_Leaky(128, (1,1))(y76)
    y76 = DarknetConv2D_BN_Leaky(256, (3,3))(y76)
    y76 = DarknetConv2D_BN_Leaky(128, (1,1))(y76)
    y76 = DarknetConv2D_BN_Leaky(256, (3,3))(y76)
    y76 = DarknetConv2D_BN_Leaky(128, (1,1))(y76)

    # PAN - path aggregation network
    #76x76 output
    y76_output = DarknetConv2D_BN_Leaky(256, (3,3))(y76)
    y76_output = DarknetConv2D(num_anchors*(num_classes+5), (1,1))(y76_output)

    #38x38 output
    y76_downsample = ZeroPadding2D(((1,0),(1,0)))(y76)
    y76_downsample = DarknetConv2D_BN_Leaky(256, (3,3), strides=(2,2))(y76_downsample)
    y38 = Concatenate()([y76_downsample, y38])
    y38 = DarknetConv2D_BN_Leaky(256, (1,1))(y38)
    y38 = DarknetConv2D_BN_Leaky(512, (3,3))(y38)
    y38 = DarknetConv2D_BN_Leaky(256, (1,1))(y38)
    y38 = DarknetConv2D_BN_Leaky(512, (3,3))(y38)
    y38 = DarknetConv2D_BN_Leaky(256, (1,1))(y38)

    y38_output = DarknetConv2D_BN_Leaky(512, (3,3))(y38)
    y38_output = DarknetConv2D(num_anchors*(num_classes+5), (1,1))(y38_output)

    #19x19 output
    y38_downsample = ZeroPadding2D(((1,0),(1,0)))(y38)
    y38_downsample = DarknetConv2D_BN_Leaky(512, (3,3), strides=(2,2))(y38_downsample)
    y19 = Concatenate()([y38_downsample, y19])
    y19 = DarknetConv2D_BN_Leaky(512, (1,1))(y19)
    y19 = DarknetConv2D_BN_Leaky(1024, (3,3))(y19)
    y19 = DarknetConv2D_BN_Leaky(512, (1,1))(y19)
    y19 = DarknetConv2D_BN_Leaky(1024, (3,3))(y19)
    y19 = DarknetConv2D_BN_Leaky(512, (1,1))(y19)

    y19_output = DarknetConv2D_BN_Leaky(1024, (3,3))(y19)
    y19_output = DarknetConv2D(num_anchors*(num_classes+5), (1,1))(y19_output)

    # 19=large, 38=medium, 76=small
    yolo4_model = Model(inputs, [y19_output, y38_output, y76_output])

    return yolo4_model

**YOLO loss**

In [19]:
def bbox_ciou(boxes1, boxes2):
    '''
    caluate ciou = iou - p2/c2 - av
    : (batch_size, grid_r, grid_c, anchor, box)
    :param boxes1: (8, 13, 13, 3, 4)   pred_xywh
    :param boxes2: (8, 13, 13, 3, 4)   label_xywh
    :return:

    For example, assume that the shapes of pred_xywh and label_xywh are both (1, 4)
    '''

    # convert to the upper left corner coordinate, the lower right corner coordinate
    boxes1_x0y0x1y1 = tf.concat([boxes1[..., :2] - boxes1[..., 2:] * 0.5,
                                 boxes1[..., :2] + boxes1[..., 2:] * 0.5], axis=-1)
    boxes2_x0y0x1y1 = tf.concat([boxes2[..., :2] - boxes2[..., 2:] * 0.5,
                                 boxes2[..., :2] + boxes2[..., 2:] * 0.5], axis=-1)
    '''
    Compare boxes1_x0y0x1y1[..., :2] and boxes1_x0y0x1y1[..., 2:] position by position, that is, compare [x0, y0] and [x1, y1] position by position, leaving the smaller ones.
    For example, leaving [x0, y0]
    This step is to avoid that w h is a negative number at the beginning, causing x0y0 to become the coordinates of the lower right corner and x1y1 to become the coordinates of the upper left corner.
    '''
    boxes1_x0y0x1y1 = tf.concat([tf.minimum(boxes1_x0y0x1y1[..., :2], boxes1_x0y0x1y1[..., 2:]),
                                 tf.maximum(boxes1_x0y0x1y1[..., :2], boxes1_x0y0x1y1[..., 2:])], axis=-1)
    boxes2_x0y0x1y1 = tf.concat([tf.minimum(boxes2_x0y0x1y1[..., :2], boxes2_x0y0x1y1[..., 2:]),
                                 tf.maximum(boxes2_x0y0x1y1[..., :2], boxes2_x0y0x1y1[..., 2:])], axis=-1)

    # The area of the two rectangles
    boxes1_area = (boxes1_x0y0x1y1[..., 2] - boxes1_x0y0x1y1[..., 0]) * (
                boxes1_x0y0x1y1[..., 3] - boxes1_x0y0x1y1[..., 1])
    boxes2_area = (boxes2_x0y0x1y1[..., 2] - boxes2_x0y0x1y1[..., 0]) * (
                boxes2_x0y0x1y1[..., 3] - boxes2_x0y0x1y1[..., 1])

    # The coordinates of the upper left corner and the lower right corner of the intersecting rectangle, the shapes are both (8, 13, 13, 3, 2)
    left_up = tf.maximum(boxes1_x0y0x1y1[..., :2], boxes2_x0y0x1y1[..., :2])
    right_down = tf.minimum(boxes1_x0y0x1y1[..., 2:], boxes2_x0y0x1y1[..., 2:])

    # The area of the intersecting rectangle inter_area. iou
    inter_section = tf.maximum(right_down - left_up, 0.0)
    inter_area = inter_section[..., 0] * inter_section[..., 1]
    union_area = boxes1_area + boxes2_area - inter_area
    iou = inter_area / (union_area + K.epsilon())

    # The coordinates of the upper left corner and the lower right corner of the enclosing rectangle, the shape is (8, 13, 13, 3, 2)
    enclose_left_up = tf.minimum(boxes1_x0y0x1y1[..., :2], boxes2_x0y0x1y1[..., :2])
    enclose_right_down = tf.maximum(boxes1_x0y0x1y1[..., 2:], boxes2_x0y0x1y1[..., 2:])

    # DIoU 계산

    # The square of the diagonal of the enclosing rectangle
    enclose_wh = enclose_right_down - enclose_left_up
    enclose_c2 = K.pow(enclose_wh[..., 0], 2) + K.pow(enclose_wh[..., 1], 2)
    
    # The square of the distance between the center points of the two rectangles
    p2 = K.pow(boxes1[..., 0] - boxes2[..., 0], 2) + K.pow(boxes1[..., 1] - boxes2[..., 1], 2)

    # Increase av. Add division by 0 protection to prevent nan.
    atan1 = tf.atan(boxes1[..., 2] / (boxes1[..., 3] + K.epsilon()))
    atan2 = tf.atan(boxes2[..., 2] / (boxes2[..., 3] + K.epsilon()))
    v = 4.0 * K.pow(atan1 - atan2, 2) / (math.pi ** 2)
    a = v / (1 - iou + v)

    # ciou loss = 1 - ciou
    ciou = iou - 1.0 * p2 / enclose_c2 - 1.0 * a * v
    return ciou

def bbox_iou(boxes1, boxes2):
    '''
    Measuring frame boxes1 (?, grid_h, grid_w, 3, 1, 4), the output of the neural network (tx, ty, tw, th) is obtained by post-processing (bx, by, bw, bh)
    All gt boxes2 in the picture (?, 1, 1, 1, 150, 4)
    '''
    boxes1_area = boxes1[..., 2] * boxes1[..., 3]  # Area of 3 prediction boxes of all grids
    boxes2_area = boxes2[..., 2] * boxes2[..., 3]  # The area of all ground truth

    # (x, y, w, h) to (x0, y0, x1, y1)
    boxes1 = tf.concat([boxes1[..., :2] - boxes1[..., 2:] * 0.5,
                        boxes1[..., :2] + boxes1[..., 2:] * 0.5], axis=-1)
    boxes2 = tf.concat([boxes2[..., :2] - boxes2[..., 2:] * 0.5,
                        boxes2[..., :2] + boxes2[..., 2:] * 0.5], axis=-1)

    # 3 prediction boxes with grids and 150 ground truth calculation iou respectively. So the shape of left_up and right_down = (?, grid_h, grid_w, 3, 150, 2)
    left_up = tf.maximum(boxes1[..., :2], boxes2[..., :2])  # The coordinates of the upper left corner of the intersecting rectangle
    right_down = tf.minimum(boxes1[..., 2:], boxes2[..., 2:])  # The coordinates of the lower right corner of the intersecting rectangle

    inter_section = tf.maximum(right_down - left_up, 0.0)  # The w and h of the intersecting rectangle are 0 when they are negative (?, grid_h, grid_w, 3, 150, 2)
    inter_area = inter_section[..., 0] * inter_section[..., 1]  # Area of the intersecting rectangle (?, grid_h, grid_w, 3, 150)
    union_area = boxes1_area + boxes2_area - inter_area  # union_area      (?, grid_h, grid_w, 3, 150)
    iou = 1.0 * inter_area / union_area  # iou                             (?, grid_h, grid_w, 3, 150)
    return iou

def loss_layer(conv, pred, label, bboxes, stride, num_class, iou_loss_thresh):
    conv_shape = tf.shape(conv)
    batch_size = conv_shape[0]
    output_size = conv_shape[1]
    input_size = stride * output_size
    conv = tf.reshape(conv, (batch_size, output_size, output_size,
                             3, 5 + num_class))
    conv_raw_prob = conv[:, :, :, :, 5:]

    pred_xywh = pred[:, :, :, :, 0:4]
    pred_conf = pred[:, :, :, :, 4:5]

    label_xywh = label[:, :, :, :, 0:4]
    respond_bbox = label[:, :, :, :, 4:5]
    label_prob = label[:, :, :, :, 5:]

    ciou = tf.expand_dims(bbox_ciou(pred_xywh, label_xywh), axis=-1)  # (8, 13, 13, 3, 1)
    input_size = tf.cast(input_size, tf.float32)

    # The weight of each prediction box xxxiou_loss = 2-(ground truth area/picture area)
    bbox_loss_scale = 2.0 - 1.0 * label_xywh[:, :, :, :, 2:3] * label_xywh[:, :, :, :, 3:4] / (input_size ** 2)
    ciou_loss = respond_bbox * bbox_loss_scale * (1 - ciou) # 1. respond_bbox is used as a mask, xxxiou_loss is calculated only when there is an object

    # 2. respond_bbox is used as a mask, and the category loss is calculated when there is an object
    prob_loss = respond_bbox * tf.nn.sigmoid_cross_entropy_with_logits(labels=label_prob, logits=conv_raw_prob)

    # 3. xxxiou_loss and category loss are relatively simple. The important thing is conf_loss, which is a focal_loss
    # There are two steps: the first step is to determine which grid_h * grid_w * 3 prediction boxes are used as counterexamples; the second step is to calculate focal_loss.

    expand_pred_xywh = pred_xywh[:, :, :, :, np.newaxis, :]  # Expand to (?, grid_h, grid_w, 3,   1, 4)
    expand_bboxes = bboxes[:, np.newaxis, np.newaxis, np.newaxis, :, :]  # Expand to (?,      1,      1, 1, 150, 4)
    iou = bbox_iou(expand_pred_xywh, expand_bboxes)  # The 3 prediction boxes of all grids and 150 ground truth respectively calculate iou.   (?, grid_h, grid_w, 3, 150
    max_iou = tf.expand_dims(tf.reduce_max(iou, axis=-1), axis=-1)  # Among 150 ground truth iou, keep the largest iou. (?, grid_h, grid_w, 3, 1)

    # respond_bgd represents whether the grid_h * grid_w * 3 prediction boxes output by this branch are counterexamples (background)
    # label has an object, respond_bgd is 0. If there is no object: if the iou with a certain gt (150 in total) exceeds iou_loss_thresh, respond_bgd is 0; if the iou with all gt (150 at most) is less than iou_loss_thresh, respond_bgd is 1.
    # respond_bgd is 0 means there is an object, which is not a counterexample; the weight respond_bgd is 1 means there is no object, which is a counterexample.
    # Interestingly, due to constant updates during model training, for the same picture, the grid_h * grid_w * 3 prediction boxes (for this branch output) of the two predictions are different. These prediction boxes are used to calculate iou to determine which prediction boxes are counterexamples.
    # Instead of using a priori box of fixed size (not fixed position).
    respond_bgd = (1.0 - respond_bbox) * tf.cast(max_iou < iou_loss_thresh, tf.float32)

    # Binary cross entropy loss
    pos_loss = respond_bbox * (0 - K.log(pred_conf + K.epsilon()))
    neg_loss = respond_bgd  * (0 - K.log(1 - pred_conf + K.epsilon()))

    conf_loss = pos_loss + neg_loss
    # Looking back at respond_bgd, the iou of a certain prediction box and a certain gt exceeds iou_loss_thresh, which is not regarded as a counterexample. When participating in the "Binary Cross Entropy of Predicted Confidence Level and True Confidence Level", this box may not be a positive example (if this box is not marked as 1 in the label). This box may not participate in the calculation of confidence loss.
    # This kind of box is generally the box near the gt box, or the other two boxes of the grid where the gt box is located. It is neither a positive example nor a negative example. It does not participate in the calculation of the confidence loss. (Called ignore in the paper

    ciou_loss = tf.reduce_mean(tf.reduce_sum(ciou_loss, axis=[1, 2, 3, 4]))  # Each sample calculates its own ciou_loss separately, and then averages
    conf_loss = tf.reduce_mean(tf.reduce_sum(conf_loss, axis=[1, 2, 3, 4]))  # Each sample calculates its own conf_loss separately, and then averages
    prob_loss = tf.reduce_mean(tf.reduce_sum(prob_loss, axis=[1, 2, 3, 4]))  # Each sample calculates its own prob_loss separately, and then averages

    return ciou_loss, conf_loss, prob_loss

# YOLO의 모델 출력 결과를 scale및 grid에 따라 좌표 복원하기
# conf(objectness)와 prob(class probability)에는 sigmoid activation function 적용
# 전체가 convolution이라서 부분적으로 activation을 후처리함.

def decode(conv_output, anchors, stride, num_class):
    conv_shape       = tf.shape(conv_output)
    batch_size       = conv_shape[0]
    output_size      = conv_shape[1]
    anchor_per_scale = len(anchors)
    conv_output = tf.reshape(conv_output, (batch_size, output_size, output_size, anchor_per_scale, 5 + num_class))
    conv_raw_dxdy = conv_output[:, :, :, :, 0:2]
    conv_raw_dwdh = conv_output[:, :, :, :, 2:4]
    conv_raw_conf = conv_output[:, :, :, :, 4:5]
    conv_raw_prob = conv_output[:, :, :, :, 5: ]
    y = tf.tile(tf.range(output_size, dtype=tf.int32)[:, tf.newaxis], [1, output_size])
    x = tf.tile(tf.range(output_size, dtype=tf.int32)[tf.newaxis, :], [output_size, 1])
    xy_grid = tf.concat([x[:, :, tf.newaxis], y[:, :, tf.newaxis]], axis=-1)
    xy_grid = tf.tile(xy_grid[tf.newaxis, :, :, tf.newaxis, :], [batch_size, 1, 1, anchor_per_scale, 1])
    xy_grid = tf.cast(xy_grid, tf.float32)
    pred_xy = (tf.sigmoid(conv_raw_dxdy) + xy_grid) * stride
    pred_wh = (tf.exp(conv_raw_dwdh) * anchors) * stride
    pred_xywh = tf.concat([pred_xy, pred_wh], axis=-1)
    pred_conf = tf.sigmoid(conv_raw_conf)
    pred_prob = tf.sigmoid(conv_raw_prob)
    return tf.concat([pred_xywh, pred_conf, pred_prob], axis=-1)

# 전체 loss 계산 함수

def yolo_loss(args, num_classes, iou_loss_thresh, anchors):
    conv_lbbox = args[0]   # (?, ?, ?, 3*(num_classes+5))
    conv_mbbox = args[1]   # (?, ?, ?, 3*(num_classes+5))
    conv_sbbox = args[2]   # (?, ?, ?, 3*(num_classes+5))
    label_sbbox = args[3]   # (?, ?, ?, 3, num_classes+5)
    label_mbbox = args[4]   # (?, ?, ?, 3, num_classes+5)
    label_lbbox = args[5]   # (?, ?, ?, 3, num_classes+5)
    true_sbboxes = args[6]   # (?, 150, 4)
    true_mbboxes = args[7]   # (?, 150, 4)
    true_lbboxes = args[8]   # (?, 150, 4)
    pred_sbbox = decode(conv_sbbox, anchors[0], 8, num_classes)
    pred_mbbox = decode(conv_mbbox, anchors[1], 16, num_classes)
    pred_lbbox = decode(conv_lbbox, anchors[2], 32, num_classes)
    sbbox_ciou_loss, sbbox_conf_loss, sbbox_prob_loss = loss_layer(conv_sbbox, pred_sbbox, label_sbbox, true_sbboxes, 8, num_classes, iou_loss_thresh)
    mbbox_ciou_loss, mbbox_conf_loss, mbbox_prob_loss = loss_layer(conv_mbbox, pred_mbbox, label_mbbox, true_mbboxes, 16, num_classes, iou_loss_thresh)
    lbbox_ciou_loss, lbbox_conf_loss, lbbox_prob_loss = loss_layer(conv_lbbox, pred_lbbox, label_lbbox, true_lbboxes, 32, num_classes, iou_loss_thresh)

    ciou_loss = sbbox_ciou_loss + mbbox_ciou_loss + lbbox_ciou_loss
    conf_loss = sbbox_conf_loss + mbbox_conf_loss + lbbox_conf_loss
    prob_loss = sbbox_prob_loss + mbbox_prob_loss + lbbox_prob_loss

    loss = ciou_loss + conf_loss + prob_loss

    loss = tf.compat.v1.Print(loss, [loss, ciou_loss, conf_loss, prob_loss], message=' loss: ')
    #tf.print([loss, ciou_loss, conf_loss, prob_loss])

    return loss

**YOLO 전체 모델 생성**

In [20]:
def create_model(input_shape, anchors_stride_base, num_classes, load_pretrained=True, freeze_body=2,
            weights_path='model_data/yolo_weights.h5'):
    '''create the training model'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors_stride_base)

    max_bbox_per_scale = 150
    iou_loss_thresh = 0.7

    model_body = yolo4_body(image_input, num_anchors, num_classes)
    print('Create YOLOv4 model with {} anchors and {} classes.'.format(num_anchors*3, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze darknet53 body or freeze all but 3 output layers.
            num = (250, len(model_body.layers)-3)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    y_true = [
        layers.Input(name='input_2', shape=(None, None, 3, (num_classes + 5))),  # label_sbbox
        layers.Input(name='input_3', shape=(None, None, 3, (num_classes + 5))),  # label_mbbox
        layers.Input(name='input_4', shape=(None, None, 3, (num_classes + 5))),  # label_lbbox
        layers.Input(name='input_5', shape=(max_bbox_per_scale, 4)),             # true_sbboxes
        layers.Input(name='input_6', shape=(max_bbox_per_scale, 4)),             # true_mbboxes
        layers.Input(name='input_7', shape=(max_bbox_per_scale, 4))              # true_lbboxes
    ]
    loss_list = layers.Lambda(yolo_loss, name='yolo_loss',
                           arguments={'num_classes': num_classes, 'iou_loss_thresh': iou_loss_thresh,
                                      'anchors': anchors_stride_base})([*model_body.output, *y_true])

    model = Model([model_body.input, *y_true], loss_list)
    #model.summary()

    return model, model_body

**class와 anchor 설정파일 읽어오기**

In [21]:
!cat model_data/voc_classes.txt
!cat model_data/yolo4_anchors.txt

aeroplane
bicycle
bird
boat
bottle
bus
car
cat
chair
cow
diningtable
dog
horse
motorbike
person
pottedplant
sheep
sofa
train
tvmonitor
12, 16,  19, 36,  40, 28,  36, 75,  76, 55,  72, 146,  142, 110,  192, 243,  459, 401

In [ ]:
def get_classes(classes_path):
    '''loads the classes'''
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

**Data augmentation modules**

In [22]:
def random_fill(image, bboxes):
    if random.random() < 0.5:
        h, w, _ = image.shape
        # Fill the black border horizontally to train small target detection
        if random.random() < 0.5:
            dx = random.randint(int(0.5*w), int(1.5*w))
            black_1 = np.zeros((h, dx, 3), dtype='uint8')
            black_2 = np.zeros((h, dx, 3), dtype='uint8')
            image = np.concatenate([black_1, image, black_2], axis=1)
            bboxes[:, [0, 2]] += dx
        # Fill the black edges vertically to train small target detection
        else:
            dy = random.randint(int(0.5*h), int(1.5*h))
            black_1 = np.zeros((dy, w, 3), dtype='uint8')
            black_2 = np.zeros((dy, w, 3), dtype='uint8')
            image = np.concatenate([black_1, image, black_2], axis=0)
            bboxes[:, [1, 3]] += dy
    return image, bboxes

def random_horizontal_flip(image, bboxes):
    if random.random() < 0.5:
        _, w, _ = image.shape
        image = image[:, ::-1, :]
        bboxes[:, [0,2]] = w - bboxes[:, [2,0]]
    return image, bboxes

def random_crop(image, bboxes):
    if random.random() < 0.5:
        h, w, _ = image.shape
        max_bbox = np.concatenate([np.min(bboxes[:, 0:2], axis=0), np.max(bboxes[:, 2:4], axis=0)], axis=-1)

        max_l_trans = max_bbox[0]
        max_u_trans = max_bbox[1]
        max_r_trans = w - max_bbox[2]
        max_d_trans = h - max_bbox[3]

        crop_xmin = max(0, int(max_bbox[0] - random.uniform(0, max_l_trans)))
        crop_ymin = max(0, int(max_bbox[1] - random.uniform(0, max_u_trans)))
        crop_xmax = max(w, int(max_bbox[2] + random.uniform(0, max_r_trans)))
        crop_ymax = max(h, int(max_bbox[3] + random.uniform(0, max_d_trans)))

        image = image[crop_ymin : crop_ymax, crop_xmin : crop_xmax]

        bboxes[:, [0, 2]] = bboxes[:, [0, 2]] - crop_xmin
        bboxes[:, [1, 3]] = bboxes[:, [1, 3]] - crop_ymin
    return image, bboxes

def random_translate(image, bboxes):
    if random.random() < 0.5:
        h, w, _ = image.shape
        max_bbox = np.concatenate([np.min(bboxes[:, 0:2], axis=0), np.max(bboxes[:, 2:4], axis=0)], axis=-1)

        max_l_trans = max_bbox[0]
        max_u_trans = max_bbox[1]
        max_r_trans = w - max_bbox[2]
        max_d_trans = h - max_bbox[3]

        tx = random.uniform(-(max_l_trans - 1), (max_r_trans - 1))
        ty = random.uniform(-(max_u_trans - 1), (max_d_trans - 1))

        M = np.array([[1, 0, tx], [0, 1, ty]])
        image = cv2.warpAffine(image, M, (w, h))

        bboxes[:, [0, 2]] = bboxes[:, [0, 2]] + tx
        bboxes[:, [1, 3]] = bboxes[:, [1, 3]] + ty
    return image, bboxes

def image_preprocess(image, target_size, gt_boxes):
    # The images passed in for training are in rgb format
    ih, iw = target_size
    h, w = image.shape[:2]
    interps = [   # Randomly choose an interpolation method
        cv2.INTER_NEAREST,
        cv2.INTER_LINEAR,
        cv2.INTER_AREA,
        cv2.INTER_CUBIC,
        cv2.INTER_LANCZOS4,
    ]
    method = np.random.choice(interps)   # Randomly choose an interpolation method
    scale_x = float(iw) / w
    scale_y = float(ih) / h
    image = cv2.resize(image, None, None, fx=scale_x, fy=scale_y, interpolation=method)

    pimage = image.astype(np.float32) / 255.
    if gt_boxes is None:
        return pimage
    else:
        gt_boxes[:, [0, 2]] = gt_boxes[:, [0, 2]] * scale_x
        gt_boxes[:, [1, 3]] = gt_boxes[:, [1, 3]] * scale_y
        return pimage, gt_boxes

def parse_annotation(annotation, train_input_size, annotation_type):
    line = annotation.split()
    image_path = line[0]
    if not os.path.exists(image_path):
        raise KeyError("%s does not exist ... " %image_path)
    image = np.array(cv2.imread(image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # No items are marked, that is, each grid is treated as a background
    exist_boxes = True
    if len(line) == 1:
        bboxes = np.array([[10, 10, 101, 103, 0]])
        exist_boxes = False
    else:
        bboxes = np.array([list(map(lambda x: int(float(x)), box.split(','))) for box in line[1:]])
    if annotation_type == 'train':
        # image, bboxes = random_fill(np.copy(image), np.copy(bboxes))    # Open when the dataset lacks small objects
        image, bboxes = random_horizontal_flip(np.copy(image), np.copy(bboxes))
        image, bboxes = random_crop(np.copy(image), np.copy(bboxes))
        image, bboxes = random_translate(np.copy(image), np.copy(bboxes))
    image, bboxes = image_preprocess(np.copy(image), [train_input_size, train_input_size], np.copy(bboxes))
    return image, bboxes, exist_boxes

**Data generator**

In [23]:
def data_generator(annotation_lines, batch_size, anchors, num_classes, max_bbox_per_scale, annotation_type):
    '''data generator for fit_generator'''
    n = len(annotation_lines)
    i = 0
    # Multi-scale training
    train_input_sizes = [320, 352, 384, 416, 448, 480, 512, 544, 576, 608]
    strides = np.array([8, 16, 32])

    while True:
        train_input_size = random.choice(train_input_sizes)

        # Number of output grids
        train_output_sizes = train_input_size // strides

        batch_image = np.zeros((batch_size, train_input_size, train_input_size, 3))

        batch_label_sbbox = np.zeros((batch_size, train_output_sizes[0], train_output_sizes[0],
                                      3, 5 + num_classes))
        batch_label_mbbox = np.zeros((batch_size, train_output_sizes[1], train_output_sizes[1],
                                      3, 5 + num_classes))
        batch_label_lbbox = np.zeros((batch_size, train_output_sizes[2], train_output_sizes[2],
                                      3, 5 + num_classes))

        batch_sbboxes = np.zeros((batch_size, max_bbox_per_scale, 4))
        batch_mbboxes = np.zeros((batch_size, max_bbox_per_scale, 4))
        batch_lbboxes = np.zeros((batch_size, max_bbox_per_scale, 4))

        for num in range(batch_size):
            if i == 0:
                np.random.shuffle(annotation_lines)

            image, bboxes, exist_boxes = parse_annotation(annotation_lines[i], train_input_size, annotation_type)
            # 각각의 앵커에 맞는 사전 작업
            label_sbbox, label_mbbox, label_lbbox, sbboxes, mbboxes, lbboxes = preprocess_true_boxes(bboxes, train_output_sizes, strides, num_classes, max_bbox_per_scale, anchors)

            batch_image[num, :, :, :] = image
            if exist_boxes:
                batch_label_sbbox[num, :, :, :, :] = label_sbbox
                batch_label_mbbox[num, :, :, :, :] = label_mbbox
                batch_label_lbbox[num, :, :, :, :] = label_lbbox
                batch_sbboxes[num, :, :] = sbboxes
                batch_mbboxes[num, :, :] = mbboxes
                batch_lbboxes[num, :, :] = lbboxes
            i = (i + 1) % n
        yield [batch_image, batch_label_sbbox, batch_label_mbbox, batch_label_lbbox, batch_sbboxes, batch_mbboxes, batch_lbboxes], np.zeros(batch_size)

def data_generator_wrapper(annotation_lines, batch_size, anchors, num_classes, max_bbox_per_scale, annotation_type):
    n = len(annotation_lines)
    if n==0 or batch_size<=0: return None
    return data_generator(annotation_lines, batch_size, anchors, num_classes, max_bbox_per_scale, annotation_type)

**Ground Truth box 처리**

In [24]:
# IOU 계산
def bbox_iou_data(boxes1, boxes2):
    boxes1 = np.array(boxes1)
    boxes2 = np.array(boxes2)
    boxes1_area = boxes1[..., 2] * boxes1[..., 3]
    boxes2_area = boxes2[..., 2] * boxes2[..., 3]
    # x1,y1,x2,y2 ==> x,y,w,h
    boxes1 = np.concatenate([boxes1[..., :2] - boxes1[..., 2:] * 0.5,
                            boxes1[..., :2] + boxes1[..., 2:] * 0.5], axis=-1)
    boxes2 = np.concatenate([boxes2[..., :2] - boxes2[..., 2:] * 0.5,
                            boxes2[..., :2] + boxes2[..., 2:] * 0.5], axis=-1)
    # 교집합의 좌상
    left_up = np.maximum(boxes1[..., :2], boxes2[..., :2])
    # 교집합의 우하
    right_down = np.minimum(boxes1[..., 2:], boxes2[..., 2:])
    inter_section = np.maximum(right_down - left_up, 0.0)
    inter_area = inter_section[..., 0] * inter_section[..., 1]
    union_area = boxes1_area + boxes2_area - inter_area
    return inter_area / union_area

# GT boxes를 anchor로 scale을 찾고 해당 scale의 grid와 anchor에 저장
def preprocess_true_boxes(bboxes, train_output_sizes, strides, num_classes, max_bbox_per_scale, anchors):
    label = [np.zeros((train_output_sizes[i], train_output_sizes[i], 3,
                       5 + num_classes)) for i in range(3)]
    bboxes_xywh = [np.zeros((max_bbox_per_scale, 4)) for _ in range(3)]
    bbox_count = np.zeros((3,))
    for bbox in bboxes:
        bbox_coor = bbox[:4]
        bbox_class_ind = bbox[4]
        onehot = np.zeros(num_classes, dtype=np.float)
        onehot[bbox_class_ind] = 1.0
        bbox_xywh = np.concatenate([(bbox_coor[2:] + bbox_coor[:2]) * 0.5, bbox_coor[2:] - bbox_coor[:2]], axis=-1)
        bbox_xywh_scaled = 1.0 * bbox_xywh[np.newaxis, :] / strides[:, np.newaxis]
        # 3단계 크기 중에서 anchor와 가장 매칭되는 곳 찾기
        iou = []
        for i in range(3):
            anchors_xywh = np.zeros((3, 4))
            anchors_xywh[:, 0:2] = np.floor(bbox_xywh_scaled[i, 0:2]).astype(np.int32) + 0.5
            anchors_xywh[:, 2:4] = anchors[i]
            iou_scale = bbox_iou_data(bbox_xywh_scaled[i][np.newaxis, :], anchors_xywh)
            iou.append(iou_scale)
        best_anchor_ind = np.argmax(np.array(iou).reshape(-1), axis=-1)
        best_detect = int(best_anchor_ind / 3)
        best_anchor = int(best_anchor_ind % 3)
        xind, yind = np.floor(bbox_xywh_scaled[best_detect, 0:2]).astype(np.int32)
        # Prevent crossing
        grid_r = label[best_detect].shape[0]
        grid_c = label[best_detect].shape[1]
        xind = max(0, xind)
        yind = max(0, yind)
        xind = min(xind, grid_r-1)
        yind = min(yind, grid_c-1)
        # bbox[4], objectness, class GT
        label[best_detect][yind, xind, best_anchor, :] = 0
        label[best_detect][yind, xind, best_anchor, 0:4] = bbox_xywh
        label[best_detect][yind, xind, best_anchor, 4:5] = 1.0
        label[best_detect][yind, xind, best_anchor, 5:] = onehot
        bbox_ind = int(bbox_count[best_detect] % max_bbox_per_scale)
        bboxes_xywh[best_detect][bbox_ind, :4] = bbox_xywh
        bbox_count[best_detect] += 1
    label_sbbox, label_mbbox, label_lbbox = label
    sbboxes, mbboxes, lbboxes = bboxes_xywh
    return label_sbbox, label_mbbox, label_lbbox, sbboxes, mbboxes, lbboxes

**학습하기**

In [25]:
import math
import random
import os
import cv2

from tensorflow.keras.layers import Input, Lambda
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

from callback_eval import Evaluate

In [26]:
def _main():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
      try:
        for gpu in gpus:
          tf.config.experimental.set_memory_growth(gpu, True)
      except RuntimeError as e:
        print(e)

    print('Please visit https://github.com/miemie2013/Keras-YOLOv4 for more complete model!')

    annotation_train_path = '2012_train.txt'
    annotation_val_path = '2012_val.txt'
    #annotation_train_path = '2007_train.txt'
    #annotation_val_path = '2007_val.txt'
    log_dir = 'logs/000/'
    classes_path = 'model_data/voc_classes.txt'
    anchors_path = 'model_data/yolo4_anchors.txt'
    class_names = get_classes(classes_path)
    num_classes = len(class_names)
    class_index = ['{}'.format(i) for i in range(num_classes)]
    anchors = get_anchors(anchors_path)

    max_bbox_per_scale = 150

    anchors_stride_base = np.array([
        [[12, 16], [19, 36], [40, 28]],
        [[36, 75], [76, 55], [72, 146]],
        [[142, 110], [192, 243], [459, 401]]
    ])
    # Some preprocessing
    anchors_stride_base = anchors_stride_base.astype(np.float32)
    anchors_stride_base[0] /= 8
    anchors_stride_base[1] /= 16
    anchors_stride_base[2] /= 32

    input_shape = (608, 608) # multiple of 32, hw
    model_path = 'yolo4_weight.h5'
    #model_path = 'logs/000/'+'ep001-loss5261.659.h5'
    #model_path = 'logs-neck/000/'+'ep050-loss5.966.h5' # neck for voc2012
    model, model_body = create_model(input_shape, anchors_stride_base, num_classes,
      load_pretrained=True, freeze_body=2,
      weights_path=model_path)

    logging = TensorBoard(log_dir=log_dir)
    checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}.h5',
        monitor='loss', save_weights_only=True, save_best_only=True, period=1)
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=3, verbose=1)
    early_stopping = EarlyStopping(monitor='loss', min_delta=0, patience=10, verbose=1)
    evaluation = Evaluate(model_body=model_body, anchors=anchors, class_names=class_index, score_threshold=0.05,
        tensorboard=logging, weighted_average=True, eval_file=annotation_val_path, log_dir=log_dir)

    with open(annotation_train_path) as f:
        lines_train = f.readlines()
    np.random.seed(10101)
    np.random.shuffle(lines_train)
    np.random.seed(None)
    num_train = len(lines_train)

    with open(annotation_val_path) as f:
        lines_val = f.readlines()
    np.random.seed(10101)
    np.random.shuffle(lines_val)
    np.random.seed(None)
    num_val = len(lines_val)
    # Train with frozen layers first, to get a stable loss.
    # Adjust num epochs to your dataset. This step is enough to obtain a not bad model.
    if True:
        model.compile(optimizer=Adam(lr=1e-3), loss={'yolo_loss': lambda y_true, y_pred: y_pred})

        batch_size = 16
        print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
        #model.fit_generator(data_generator_wrapper(lines_train[:num_train], batch_size, input_shape, anchors, num_classes),
        #model.fit_generator(data_generator_wrapper(lines_train, batch_size, anchors_stride_base, num_classes, max_bbox_per_scale, 'train'),
        model.fit(data_generator_wrapper(lines_train, batch_size, anchors_stride_base, num_classes, max_bbox_per_scale, 'train'),
                steps_per_epoch=max(1, num_train//batch_size),
                epochs=50,
                initial_epoch=0,
                callbacks=[logging, checkpoint, reduce_lr, early_stopping])

    # Unfreeze and continue training, to fine-tune.
    # Train longer if the result is not good.
    if True:
        for i in range(len(model.layers)):
            model.layers[i].trainable = True
        model.compile(optimizer=Adam(lr=1e-5), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
        print('Unfreeze all of the layers.')

        batch_size = 4 # note that more GPU memory is required after unfreezing the body
        #batch_size = 2 # note that more GPU memory is required after unfreezing the body
        print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
        model.fit_generator(data_generator_wrapper(lines_train, batch_size, anchors_stride_base, num_classes, max_bbox_per_scale, 'train'),
            steps_per_epoch=max(1, num_train//batch_size),
            epochs=50000,
            initial_epoch=50,
            callbacks=[logging, checkpoint, reduce_lr, early_stopping, evaluation])

    # Further training if needed.

In [27]:
!cp /mydrive/yolo4_weight.h5 .

**주의: 아래 YOLO의 학습에는 많은시간이 소요됩니다.**  
Google colab 무료 버전의 한계 때문에 colab에서 끝까지 학습하는 것은 어렵습니다.  
학습이 시작되고 정상적으로 진행되는 데 까지만 확인하도록 합시다.  
완전한 학습을 위해서는 GPU를 갖춘 local desktop 개발 환경에서 수행할 필요가 있습니다.  

In [ ]:
_main()

Physical devices cannot be modified after being initialized
Please visit https://github.com/miemie2013/Keras-YOLOv4 for more complete model!
Create YOLOv4 model with 9 anchors and 20 classes.
Load weights yolo4_weight.h5.
Freeze the first 367 layers of total 370 layers.
Instructions for updating:
Use tf.print instead of tf.Print. Note that tf.print returns a no-output operator that directly prints the output. Outside of defuns or eager mode, this operator will not be executed unless it is directly specified in session.run or used as a control dependency for other operators. This is only a concern in graph mode. Below is an example of how to ensure tf.print executes in graph mode:

Train on 5717 samples, val on 5823 samples, with batch size 16.


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/50
357/357 [==============================] - 443s 1s/step - loss: 781.0318
Epoch 2/50
357/357 [==============================] - 342s 959ms/step - loss: 86.1122
Epoch 3/50
357/357 [==============================] - 341s 958ms/step - loss: 45.7343
Epoch 4/50
357/357 [==============================] - 336s 944ms/step - loss: 30.9916
Epoch 5/50
357/357 [==============================] - 341s 955ms/step - loss: 23.7067
Epoch 6/50
357/357 [==============================] - 338s 948ms/step - loss: 19.2493
Epoch 7/50
357/357 [==============================] - 339s 950ms/step - loss: 16.4438
Epoch 8/50
357/357 [==============================] - 341s 956ms/step - loss: 14.3859
Epoch 9/50
357/357 [==============================] - 343s 960ms/step - loss: 13.0039
Epoch 10/50
357/357 [==============================] - 343s 961ms/step - loss: 11.7843
Epoch 11/50
357/357 [==============================] - 346s 970ms/step - loss: 10.9311
Epoch 12/50
357/357 [==============================] -

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 51/50000
1429/1429 [==============================] - 1536s 1s/step - loss: 6.2276

Epoch end eval mAP on weight logs/000/ep050-loss5.597.h5
Epoch 51 mAP 0.09769709155624647

Epoch 52/50000
1429/1429 [==============================] - 1456s 1s/step - loss: 6.3728

Epoch end eval mAP on weight logs/000/ep050-loss5.597.h5
Epoch 52 mAP 0.09769709155624647

Epoch 53/50000
1429/1429 [==============================] - 1447s 1s/step - loss: 6.2527

Epoch end eval mAP on weight logs/000/ep050-loss5.597.h5
Epoch 53 mAP 0.09769709155624647

Epoch 54/50000
 324/1429 [=====>........................] - ETA: 18:38 - loss: 6.0620

**추가 과제**  
`predict_video.py` 파일을 이용하여, 동영상 인식 및 저장 기능이 있으니,  
video 파일의 객체 인식 결과를 확인해 보세요.

In [54]:
class Mish(Layer):
    '''
    Mish Activation Function.
    .. math::
        mish(x) = x * tanh(softplus(x)) = x * tanh(ln(1 + e^{x}))
    Shape:
        - Input: Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
        - Output: Same shape as the input.
    Examples:
        >>> X_input = Input(input_shape)
        >>> X = Mish()(X_input)
    '''

    def __init__(self, **kwargs):
        super(Mish, self).__init__(**kwargs)
        self.supports_masking = True

    def call(self, inputs):
        return inputs * K.tanh(K.softplus(inputs))

    def get_config(self):
        config = super(Mish, self).get_config()
        return config

    def compute_output_shape(self, input_shape):
        return input_shape

In [47]:
from keras.layers.pooling import GlobalAveragePooling2D
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
from keras.layers import Input, GlobalAveragePooling2D, BatchNormalization, Conv2D, MaxPooling2D, Activation, Dense, Flatten, Reshape
from keras.models import Model
from keras.optimizer_v2.adam import Adam



(x_train, y_train), (x_test, y_test) = mnist.load_data()

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
x_train.shape, y_train.shape

((60000, 28, 28), (60000, 10))

In [73]:
inputs = Input(shape=(28, 28))
x = Reshape(input_shape=(28, 28), target_shape=(28, 28, 1))(inputs)
x = Conv2D(32, (5, 5), padding='same')(x)
x = BatchNormalization()(x)
x = Mish()(x)
x = MaxPooling2D()(x)

x = Conv2D(64, (3, 3), padding='valid')(x)
x = BatchNormalization()(x)
x = Mish()(x)
x = MaxPooling2D()(x)

x = Conv2D(5, (3, 3), 1, padding='valid')(x)
x = BatchNormalization()(x)
x = Mish()(x)
x = MaxPooling2D()(x)

x = Conv2D(1, (2, 2), 1)(x)
x = GlobalAveragePooling2D()(x)
# x = Flatten()(x)
outputs = Dense(10, activation='softmax')(x)

model = Model(inputs, outputs)

# model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 28, 28)]          0         
                                                                 
 reshape_15 (Reshape)        (None, 28, 28, 1)         0         
                                                                 
 conv2d_43 (Conv2D)          (None, 28, 28, 32)        832       
                                                                 
 batch_normalization_28 (Bat  (None, 28, 28, 32)       128       
 chNormalization)                                                
                                                                 
 mish_22 (Mish)              (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 14, 14, 32)       0         
 g2D)                                                      

In [85]:
model = keras.Sequential(
    [
        keras.Input(shape=(28, 28)),
        Reshape(input_shape=(28, 28), target_shape=(28, 28, 1)),
        layers.Conv2D(32, kernel_size=(3, 3)),
     BatchNormalization(),
     Mish(),
        layers.MaxPooling2D(pool_size=(2, 2)),
     
        layers.Conv2D(64, kernel_size=(3, 3)),
     BatchNormalization(),
     Mish(),
        layers.MaxPooling2D(pool_size=(2, 2)),
     
        Conv2D(1, kernel_size=(3, 3)),
        # layers.GlobalAveragePooling2D(),
     Flatten(),
        layers.Dropout(0.5),
        layers.Dense(10, activation="softmax"),
    ]
)

In [86]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy',
              metrics=['accuracy'])

In [87]:
history = model.fit(x_train, y_train, 
                    batch_size=64, epochs=12, 
                    validation_data=(x_test, y_test), validation_split=0.3)

Epoch 1/12
657/657 [==============================] - 5s 6ms/step - loss: 1.3586 - accuracy: 0.5080 - val_loss: 0.4453 - val_accuracy: 0.8871
Epoch 2/12
657/657 [==============================] - 4s 7ms/step - loss: 0.8252 - accuracy: 0.6974 - val_loss: 0.3566 - val_accuracy: 0.8920
Epoch 3/12
657/657 [==============================] - 4s 7ms/step - loss: 0.7194 - accuracy: 0.7300 - val_loss: 0.2588 - val_accuracy: 0.9323
Epoch 4/12
657/657 [==============================] - 4s 7ms/step - loss: 0.6758 - accuracy: 0.7456 - val_loss: 0.2129 - val_accuracy: 0.9446
Epoch 5/12
657/657 [==============================] - 4s 6ms/step - loss: 0.6509 - accuracy: 0.7546 - val_loss: 0.1950 - val_accuracy: 0.9476
Epoch 6/12
657/657 [==============================] - 4s 6ms/step - loss: 0.6279 - accuracy: 0.7630 - val_loss: 0.1848 - val_accuracy: 0.9488
Epoch 7/12
657/657 [==============================] - 4s 7ms/step - loss: 0.6083 - accuracy: 0.7680 - val_loss: 0.2181 - val_accuracy: 0.9359
Epoch 

In [ ]:
result = model.predict(x[0])

np.argmax(result)